In [1]:
from IPython import get_ipython
from IPython.display import clear_output, Image, display, HTML

# Exploring the Convection-Diffusion Equation
## Using the Finite Element Formulation


In this Notebook we explore the steady convection-diffusion-reaction equation:

$$
\begin{aligned}
\mathbf{\beta} \cdot \nabla \phi  - \epsilon \Delta \phi = f 
\end{aligned}
$$



The Peclet number is the ratio between the advective transport and the diffusive transport rates. The Peclet number can be expressed as the global Peclet number, respective to the whole domain, and as the local Peclet number, related to a given element of a finite element mesh. Those are defined as:
$$
\begin{aligned}
Pe &= \dfrac{||\mathbf{\beta}||}{2\epsilon}\\
Pe_{mesh} &= \dfrac{||\mathbf{\beta}||h}{2\epsilon}
\end{aligned}
$$


Problems with large $Pe$, that is, advection dominated, reveal numerical difficulties. 

<img src="Pe.png" align="center"/>

The Galerkin Method presents unstable behavior as $Pe_{mesh}$ increases.

## Weak formulation

We consider a square unit domain $\Omega$ with Dirichlet boundary conditions on $\partial \Omega$. The discretization of the domain $\Omega$ into $nel$ number of elements is such that $\Omega=\cup^{nel}_{e=1} \Omega^e$ and $\emptyset =\cap^{nel}_{e=1}\Omega^e$. Considering ${S}^h$ the space of the test functions and ${V}^h$ the space of the weight functions, the weak (variational) formulation of the CDR equation becomes: find $\phi^h(\textbf{x}) \in {S}^h$ such that $\forall w^h \in {V}^h$: 

$$
\begin{aligned}
    \int_{\Omega}\bigg(w^h(\mathbf{\beta}\cdot\nabla\phi^h) + w^h \nabla \cdot (\epsilon\nabla\phi^h)\bigg)d\Omega = \int_{\Omega}w^hfd\Omega \hspace{1cm}\forall w^h \in {V}^h
\end{aligned}
$$

Using the divergence theorem and the fact that $w^h = 0$ on $\partial \Omega$, the previous equation becomes: 
$$
\begin{aligned}
    \int_{\Omega}\bigg(w^h(\mathbf{\beta}\cdot\nabla\phi^h) + \nabla w^h  \cdot (\epsilon\nabla\phi^h) \bigg)d\Omega = \int_{\Omega}w^hfd\Omega +  \int_{\partial\Omega_N}w^hgd\Gamma \hspace{1cm}\forall w^h \in {V}^h
\end{aligned}
$$

In [2]:
get_ipython().run_line_magic('matplotlib', 'inline')
from ipywidgets import interact, interactive
from IPython.display import clear_output, display, HTML

In [3]:
from fenics import *
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import time
import warnings

warnings.filterwarnings("ignore")
set_log_level(40)


class InitialConditions(UserExpression):
    def __init__(self, **kwargs):
        random.seed(2 + MPI.rank(MPI.comm_world))
        super().__init__(**kwargs)
    def eval(self, values, x):
        values[0] = 0.0 + 0.02*(0.5 - random.random())
        values[1] = 0.0
    def value_shape(self):
        return (3,)

def cavity_boundary_condition(W):
    class Lid(SubDomain):
        def inside(self, x, on_boundary):
            return(on_boundary and near (x[1], 1.0))

    class Walls(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and (near(x[0], 0.0) or near(x[0], 1.0) or near(x[1], 0.0))

    def zero(x):
        return (near(x[0], 0.0) and near(x[1], 0.0))
    # Dirichlet value
    g1 = Expression(("exp(-(pow(x[0] - 0.5, 2)/(2*pow(0.18,2))))", 0.0), degree = 2)
    #g1 = Constant((1.0, 0.0))
    g2 = Constant((0.0, 0.0))
    g3 = Constant(0.0)
    # conditions
    bc1 = DirichletBC(W.sub(0), g1, Lid())
    bc2 = DirichletBC(W.sub(0), g2, Walls())
    bc3 = DirichletBC(W.sub(1), g3, zero, 'pointwise')
    bcs = [bc1, bc2, bc3]
    return bcs


def mesh2triang(mesh):
    xy = mesh.coordinates()
    return tri.Triangulation(xy[:, 0], xy[:, 1], mesh.cells())


def myplot(mymesh, field, N):
    #plt.gca().set_aspect('equal')
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
    ax1.triplot(mesh2triang(mymesh), color='k')
    C = field.compute_vertex_values(mymesh)
    X, Y = mymesh.coordinates()[:,0], mymesh.coordinates()[:,1]
    Cr   = C.reshape(N,N)
    plt.yticks(ticks = [])
    plt.xticks(ticks = [])
    ax2.contourf(Cr)
    mappable =  matplotlib.cm.ScalarMappable(norm=None, cmap=None)
    mappable.set_array(C)
    fig.colorbar(mappable)



def cdr2d(N , Re , SUPG ):
    VTK_Output                  = True                        # True = VTK / False = HDF5
    SUPG                        = True    
    Directory                   = "/mnt/c/temp/Results_NS/"             # "/mnt/c/temp/Results_NS/"
    Directory_PP                = "/mnt/c/temp/"
    Directory_CD                = "/mnt/c/temp/Results_CD/"             # "/mnt/c/temp/Results_NS/"
    Reynolds_Ramp               = True
    Iterative_Solver_NS         = True                        # True = GMRES / False = LU
    Problem_NS                  = "Cavity"
    SUPS                        = True                        # True = SUPS / False = Taylor-Hood    
    LSIC                        = True 

    parameters["form_compiler"]["optimize"]     = True
    parameters["form_compiler"]["cpp_optimize"] = True

    Fr                          = 1.0
    Froude_inv                  = Constant(1.0/pow(Fr,2))
    f                           = Froude_inv*Constant((0.0,0.0))
    nu                          = Constant(0.0)
    if Re != 0:
        nu                          = Constant(1/Re) 

    mesh        = UnitSquareMesh(N-1, N-1)
    P1v         = VectorElement("Lagrange", mesh.ufl_cell(), 2)
    parameters["form_compiler"]["quadrature_degree"] = 3
    P1          = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
    TH          = P1v*P1
    W           = FunctionSpace(mesh, TH)
    vp          = TrialFunction(W)
    (v, p)      = split(vp)
    (w, q)      = TestFunctions(W)
    vp_         = Function(W)
    v_, p_      = split(vp_)
    u_init = InitialConditions()
    vp_.interpolate(u_init)
    vn          = Function(W.sub(0).collapse())
    vnn         = Function(W.sub(0).collapse())
    bcs         = cavity_boundary_condition(W)
    h           = CellDiameter(mesh)
    vnorm       = sqrt(dot(v_, v_))
    F           = (inner(dot(v_,nabla_grad(v)), w) + nu*inner(grad(w), grad(v)) - inner(p,div(w)) + inner(q,div(v)) - inner(f, w))*dx
    R           = dot(v_,nabla_grad(v)) - nu*div(grad(v)) + grad(p) - f
    tau         = ((2.0*vnorm/h)**2 + 9.0*(4.0*nu/(h*2))**2)**(-0.5)
    vnorm       = sqrt(dot(v_, v_))
    F_supg      = tau*inner(R, dot(v_, nabla_grad(w)))*dx

    if SUPG == True:
        F      += F_supg

    F1          = action(F, vp_)
    J           = derivative(F1, vp_, vp)

    problem = NonlinearVariationalProblem(F1, vp_, bcs, J)
    solver  = NonlinearVariationalSolver(problem)
    prm = solver.parameters
    prm["nonlinear_solver"]                         ="newton"
    prm["newton_solver"]["absolute_tolerance"]      = 1E-6
    prm["newton_solver"]["relative_tolerance"]      = 1E-6
    prm["newton_solver"]["convergence_criterion"]   = "incremental"
    prm["newton_solver"]["maximum_iterations"]      = 20
    prm["newton_solver"]["relaxation_parameter"]    = 0.9
    prm["newton_solver"]["linear_solver"]           = "direct"
    prm["newton_solver"]["error_on_nonconvergence"] = False

    if Iterative_Solver_NS == True:
        prm["newton_solver"]["linear_solver"] = "gmres"
        prm["newton_solver"]["krylov_solver"]["absolute_tolerance"] = 1E-5
        prm["newton_solver"]["krylov_solver"]["relative_tolerance"] = 1E-5
        prm["newton_solver"]["krylov_solver"]["maximum_iterations"] = 75000
        #prm["newton_solver"]["krylov_solver"]["monitor_convergence"] = True
        #prm["newton_solver"]["krylov_solver"]["nonzero_initial_guess"] = True
        #prm["newton_solver"]["krylov_solver"]["gmres"]["restart"] = 40
        prm["newton_solver"]["krylov_solver"]["error_on_nonconvergence"] = False
        prm["newton_solver"]["preconditioner"] = "none"
        #info(prm, True)
        #list_krylov_solver_preconditioners()



    converged_flag, nIter = solver.solve()

    v_ , p_ = vp_.split(deepcopy=True)
       
    myplot(mesh, p_, N)

    return 0


In [4]:
import ipywidgets as widgets
SUPG = widgets.Checkbox(
    value=False,
    description="SUPG stabilization",
    disabled=False,
    indent=True
)
Re   = widgets.FloatSlider(
    value=0,
    min=0,
    step=1.,
    max=100.,
    description= "Re"
)
N    = widgets.IntSlider(
    value=3,
    min=3,
    max=50,
    step=1,
    description="Nodes / dim"
)

w = interactive(cdr2d, N=N, Re=Re, SUPG=SUPG)
display(w)



interactive(children=(IntSlider(value=3, description='Nodes / dim', max=50, min=3), FloatSlider(value=0.0, des…